In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import re

In [2]:
!ls

DOB_Permit_Issuance.csv
Data_Cleaning.ipynb
Derived Data.ipynb
EDA.ipynb
NY Building Data Worksheet.twb
Pavan_Kumar_Boinapalli_Data_Science_Report.docx
data_cleaned.csv
data_derived.csv
nyc_cd_population.csv
nyc_population.csv.csv
~$van_Kumar_Boinapalli_Data_Science_Report.docx


In [3]:
data = pd.read_csv("DOB_Permit_Issuance.csv")

/Users/pavankumarboinapalli/DSWorkspace/ProgramFiles/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,9,10,15,25,31,33,34,35,36,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.shape

(3508249, 60)

In [5]:
data_cleaned = data.copy()

## Removing columns with More than 85% Missing Values

In [6]:
na_df = pd.DataFrame(np.round(data.isna().sum()*100/data.shape[0],2)).sort_values(0,ascending=False)
na_cols_to_remove = na_df[na_df[0]>85].index.values

for col_to_remove in na_cols_to_remove:
    data_cleaned.drop(col_to_remove,axis=1,inplace=True)
    data_cleaned.reset_index().drop('index',axis=1,inplace=True)
print("dropped columns with more than 85% missing values{}".format(na_cols_to_remove))

dropped columns with more than 85% missing values['Site Safety Mgr Business Name' "Site Safety Mgr's First Name"
 "Site Safety Mgr's Last Name" 'HIC License' 'Oil Gas'
 'Special District 2' "Permittee's Other Title" 'Special District 1']


In [7]:
data_cleaned.select_dtypes('O').columns

Index(['BOROUGH', 'Bin #', 'House #', 'Street Name', 'Job Type', 'Self_Cert',
       'Block', 'Lot', 'Community Board', 'Residential', 'Work Type',
       'Permit Status', 'Filing Status', 'Permit Type', 'Permit Subtype',
       'Site Fill', 'Filing Date', 'Issuance Date', 'Expiration Date',
       'Job Start Date', 'Permittee's First Name', 'Permittee's Last Name',
       'Permittee's Business Name', 'Permittee's Phone #',
       'Permittee's License Type', 'Permittee's License #',
       'Act as Superintendent', 'Superintendent First & Last Name',
       'Superintendent Business Name', 'Owner's Business Type', 'Non-Profit',
       'Owner's Business Name', 'Owner's First Name', 'Owner's Last Name',
       'Owner's House #', 'Owner's House Street Name', 'Owner’s House City',
       'Owner’s House State', 'Owner’s House Zip Code', 'Owner's Phone #',
       'DOBRunDate', 'NTA_NAME'],
      dtype='object')

In [8]:
num_cols = ["Bin #","Job #","Job doc. #","Block","Lot","PERMIT_SI_NO","Permit Sequence #","Community Board"]
cell_no_cols = ["Owner's Phone #","Permittee's Phone #"]
zip_cols = ["Owner’s House Zip Code","Zip Code"]
float_cols = ['Bldg Type','COUNCIL_DISTRICT','CENSUS_TRACT','Permittee\'s License #']
date_cols = ['DOBRunDate','Job Start Date','Issuance Date','Expiration Date','Filing Date']
bool_cols = ['Self_Cert','Act as Superintendent','Non-Profit']

In [9]:
def clean_numeric_columns(df,int_cols,float_cols,cell_no_cols,zip_cols):
    data_to_clean = df.copy()
    
    def is_illegal_number_drop(x):
        return (not all(char.isdigit() for char in str(x).strip()))
    
    def is_illegal_number(x):
        return re.match('^[0-9]+(\.0+)?$',str(x).strip())==None
        
    
    def is_illegal_phone_num(x):
        if len(x)==10:
            return re.match('^[0-9]+$',str(x).strip())==None
        elif len(x)>=10:
            return re.match('^[0-9]+(\.0+)?$',str(x).strip())==None
        else:
            return True
    
    def is_llegal_zip(x):
        if len(x)==5:
            return re.match('^[0-9]+$',str(x).strip())==None
        elif len(x)==7:
            return re.match('^[0-9]+(\.0+)?$',x)==None
        else:
            return True

    
    for col in int_cols:
        illegal_nums = data_to_clean[col].astype('str').apply(lambda x: is_illegal_number(x))
        illegal_row_index = np.where(illegal_nums.values)[0]
        data_to_clean.loc[illegal_row_index,col] = np.nan
        print('Replaced {} illegal values of \'{}\' column with Nan'.format(len(illegal_row_index),col))
        data_to_clean[col].fillna(0,inplace=True)
        data_to_clean[col] = data_to_clean[col].astype('int')
        
    for col in float_cols:
        illegal_floats = data_to_clean[col].astype('str').apply(lambda x: is_illegal_number(x))
        illegal_row_index = np.where(illegal_floats.values)[0]
        data_to_clean.loc[illegal_row_index,col] = np.nan
        print('Replaced {} illegal values of \'{}\' column with Nan'.format(len(illegal_row_index),col))
        data_to_clean[col].fillna(0,inplace=True)
        data_to_clean[col] = data_to_clean[col].astype('int')
        
    for col in cell_no_cols:
        illegal_cell_nums = data_to_clean[col].astype('str').apply(lambda x: is_illegal_phone_num(x))
        illegal_row_index = np.where(illegal_cell_nums.values)[0]
        data_to_clean.loc[illegal_row_index,col] = np.nan
        data_to_clean[col].fillna(0,inplace=True)
        print('Replaced {} illegal values of \'{}\' column with Nan'.format(len(illegal_row_index),col))
        data_to_clean[col] = data_to_clean[col].astype('int')
        
    for col in zip_cols:
        illegal_zips = data_to_clean[col].astype('str').apply(lambda x: is_llegal_zip(x))
        illegal_row_index = np.where(illegal_zips.values)[0]
        data_to_clean.loc[illegal_row_index,col] = np.nan
        print('Replaced {} illegal values of \'{}\' column with Nan'.format(len(illegal_row_index),col))
        data_to_clean[col].fillna(0,inplace=True)
        data_to_clean[col] = data_to_clean[col].astype('int')
        
    data_cleaned = data_to_clean.copy()
        
    return data_cleaned
    

In [10]:
def clean_bool_cols(df,bool_cols):
    data_to_clean = df.copy()
    def clean_bool_value(x):
        if x in ['Y','y','YES','Yes']:
            return 'Yes'
        elif x in ['N','No','no','n']:
            return 'No'
        else:
            return np.nan
        
    for col in bool_cols:
        data_to_clean[col] = data_to_clean[col].apply(lambda x : clean_bool_value(x))
    data_cleaned = data_to_clean.copy()
    
    return data_cleaned
        

In [11]:
data_df = clean_numeric_columns(data_cleaned,num_cols,float_cols,cell_no_cols,zip_cols)

Replaced 4 illegal values of 'Bin #' column with Nan
Replaced 0 illegal values of 'Job #' column with Nan
Replaced 0 illegal values of 'Job doc. #' column with Nan
Replaced 506 illegal values of 'Block' column with Nan
Replaced 533 illegal values of 'Lot' column with Nan
Replaced 0 illegal values of 'PERMIT_SI_NO' column with Nan
Replaced 0 illegal values of 'Permit Sequence #' column with Nan
Replaced 4766 illegal values of 'Community Board' column with Nan
Replaced 54477 illegal values of 'Bldg Type' column with Nan
Replaced 12258 illegal values of 'COUNCIL_DISTRICT' column with Nan
Replaced 12258 illegal values of 'CENSUS_TRACT' column with Nan
Replaced 241025 illegal values of 'Permittee's License #' column with Nan
Replaced 59218 illegal values of 'Owner's Phone #' column with Nan
Replaced 18006 illegal values of 'Permittee's Phone #' column with Nan
Replaced 71598 illegal values of 'Owner’s House Zip Code' column with Nan
Replaced 2281 illegal values of 'Zip Code' column with Nan

In [12]:
data_df = clean_bool_cols(data_df,bool_cols)

In [15]:
data_df['Residential'].fillna("No",inplace=True)

In [16]:
data_df.to_csv('data_cleaned.csv')